In [ ]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import palettable as pal

In [ ]:
cmaps = OrderedDict()

In [ ]:
df = pd.DataFrame()

In [ ]:
with open("2020 Outages.csv") as f:
    df = pd.read_csv(f)

In [ ]:
months = {'1': 'January', '2': 'February', '3': 'March', '4': 'April', '5': 'May', '6': 'June', '7': 'July', '8': 'August', '9': 'September', '10': 'October', '11': 'November', '12': 'December'}

In [ ]:
sum(df['SAIDI'])

In [ ]:
df['Cause'].fillna(0)

In [ ]:
mask = df['Cause'].isna()
df.loc[mask, :] = 'Undetermined'

In [ ]:
cause_set = set(df['Cause'])

In [ ]:
month = input("Enter the number for the Month: ")

In [ ]:
df2 = df
mask = []
for i, cause in enumerate(df2['Outage Date']):
    if cause[:2] == '12':
        mask.append(True)
    else:
        mask.append(False)

In [ ]:
df2 = df2.loc[mask, :]
df2 = df2.reset_index(drop=True)

In [ ]:
labels = list(cause_set)
SAIDI_cause = []
for label in labels:
    idx = df2.index[df2['Cause'] == label].tolist()
    SAIDI_cause.append(sum(df2['SAIDI'][idx]))
cause_dict = {labels[i]: SAIDI_cause[i] for i in range(len(labels))}
sorted_cause = {k: v for k, v in sorted(cause_dict.items(), key = lambda item: item[1], reverse=True)}

In [ ]:
other_values = 0
dict_del = []
for key in sorted_cause:
    if sorted_cause[key]/sum(list(sorted_cause.values())) < .03:  #Set what % to not include
        other_values = other_values + sorted_cause[key]
        dict_del.append(key)
for key in dict_del:
    del sorted_cause[key]
sorted_cause.update({"Other": other_values})

In [ ]:
mask = []
dmg_equip = df2
for i, cause in enumerate(dmg_equip['Cause']):
    if cause == 'Device/Equipment Damaged':
        mask.append(True)
    else:
        mask.append(False)
dmg_equip = dmg_equip.loc[mask, :]
dmg_equip = dmg_equip.reset_index(drop=True)

In [ ]:
device_set = set(dmg_equip['Damaged Device'])
device_list = list(device_set)
SAIDI_device = []
for label in device_list:
    idx = dmg_equip.index[dmg_equip['Damaged Device'] == label].tolist()
    SAIDI_device.append(sum(dmg_equip['SAIDI'][idx]))
device_dict = {device_list[i]: SAIDI_device[i] for i in range(len(device_list))}
sorted_device = {k: v for k, v in sorted(device_dict.items(), key = lambda item: item[1], reverse=True)}

In [ ]:
other_values = 0
dict_del = []
for key in sorted_device:
    if sorted_device[key]/sum(list(sorted_device.values())) < .03:  #Set what % to not include
        other_values = other_values + sorted_device[key]
        dict_del.append(key)
for key in dict_del:
    del sorted_device[key]
sorted_device.update({"Other": other_values})

In [ ]:
sorted_device

In [ ]:
def func(pct, allvals):
    absolute = float(pct/100.*np.sum(allvals))
    return "{:.3f}\n({:.1f}%)".format(absolute, pct) #Change how many decimals appear here

In [ ]:
fig, ax = plt.subplots(figsize=(20, 18), subplot_kw=dict(aspect="equal"))
explode = np.zeros(len(list(sorted_cause.values())))
explode[0] = 0.05
patches, texts, autotexts = plt.pie(list(sorted_cause.values()), autopct = lambda pct: func(pct, list(sorted_cause.values())), 
                                    labels = list(sorted_cause.keys()), textprops={'size': 'large'}, explode=explode, 
                                    colors=pal.cartocolors.qualitative.Antique_10.mpl_colors, startangle = 90, shadow=True)
#plt.legend(patches, list(sorted_cause.keys()), loc="upper left", prop={'size': 12})
#plt.tight_layout()
plt.setp(autotexts, size=16)
for i in range(len(texts)):
    texts[i].set_fontsize(16)
    texts[i].set_weight('bold')
title_str = months[month] + " Outage Cause Contribution"
ax.set_title(title_str, fontsize = 32, weight='bold')
plt.show

In [ ]:
fig, ax = plt.subplots(figsize=(20, 18), subplot_kw=dict(aspect="equal"))
explode = np.zeros(len(list(sorted_device.values())))
explode[0] = 0.05
patches, texts, autotexts = plt.pie(list(sorted_device.values()), 
                                    autopct = lambda pct: func(pct, list(sorted_device.values())), 
                                    labels = list(sorted_device.keys()), textprops={'size': 'large'}, 
                                    colors=pal.cartocolors.qualitative.Antique_10.mpl_colors, startangle = 90,
                                    explode = explode, shadow = True)
#plt.legend(patches, list(sorted_device.keys()), loc="upper left", prop={'size': 12})
#plt.tight_layout()
plt.setp(autotexts, size=16)
for i in range(len(texts)):
    texts[i].set_fontsize(16)
    texts[i].set_weight('bold')
title_str = months[month] + " Damaged Device Contribution"
ax.set_title(title_str, fontsize = 32, weight='bold')
plt.show